In [1]:
import os
import findspark
findspark.init(os.getenv('SPARK_HOME'))
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName("lr_example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 19:25:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/16 19:25:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], 
                            outputCol='features')

In [10]:
output = assembler.transform(data)
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
output.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [19]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [24]:
train_data, test_data = final_data.randomSplit([0.85, 0.15])

In [25]:
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                430|
|   mean| 497.19178715864626|
| stddev|  77.99687145791212|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                 70|
|   mean|  512.3507235868225|
| stddev|  86.43220196582091|
|    min|  304.1355915788555|
|    max|  744.2218671047146|
+-------+-------------------+



In [26]:
lr = LinearRegression(featuresCol="features", labelCol="Yearly Amount Spent", predictionCol="prediction")

In [28]:
lrModel = lr.fit(train_data)

22/08/16 19:38:48 WARN Instrumentation: [187a72c7] regParam is zero, which might cause numerical instability and overfitting.


In [29]:
test_results = lrModel.evaluate(test_data)

In [31]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  6.550794420875491|
|-21.670110003238392|
|-0.6110231960348074|
| 3.9894731993537107|
|  -1.67799761693243|
| -5.824845656923685|
| -2.097098186109406|
|0.08228626514107873|
|  -2.62050605770537|
| -9.217669031671562|
|-1.3482644567230864|
|   9.01564893798377|
| -5.682404919999499|
|-13.589041356989128|
|-3.1248321353075426|
|  16.37857355872069|
| 2.3898029204286786|
| -9.940068701073642|
|  12.80076955199047|
| -6.707983244748448|
+-------------------+
only showing top 20 rows



In [32]:
test_results.r2, test_results.rootMeanSquaredError, test_results.meanSquaredError, test_results.meanAbsoluteError

(0.9839637691614237, 10.86681446778234, 118.08765667720358, 8.992004205881889)

In [34]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.9716756438877...|
|[31.1239743499119...|
|[31.2606468698795...|
|[31.3584771924370...|
|[31.8627411090001...|
|[31.9453957483445...|
|[32.0775900443291...|
|[32.0961089938451...|
|[32.2620031069949...|
|[32.2784433479196...|
|[32.2952476262204...|
|[32.2996471635049...|
|[32.3326370683417...|
|[32.3810345909634...|
|[32.3862518555859...|
|[32.4042950459835...|
|[32.4495215611424...|
|[32.4612124451154...|
|[32.5332399826547...|
|[32.6055836415705...|
+--------------------+
only showing top 20 rows



In [36]:
predictions = lrModel.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.9716756438877...|488.08781533601723|
|[31.1239743499119...|508.61716384300416|
|[31.2606468698795...| 421.9376544529862|
|[31.3584771924370...| 491.1864772501217|
|[31.8627411090001...| 557.9761387909791|
|[31.9453957483445...| 662.8447695945756|
|[32.0775900443291...|403.13023340521545|
|[32.0961089938451...| 375.3161691451021|
|[32.2620031069949...|444.98362344156976|
|[32.2784433479196...|497.48796699681975|
|[32.2952476262204...|414.64426363775283|
|[32.2996471635049...| 538.0953334249623|
|[32.3326370683417...| 538.3998905961744|
|[32.3810345909634...| 546.3138468198931|
|[32.3862518555859...|421.27491323215713|
|[32.4042950459835...|430.99045364061067|
|[32.4495215611424...| 501.5885761321508|
|[32.4612124451154...| 553.2802349577746|
|[32.5332399826547...| 676.4349302097046|
|[32.6055836415705...| 511.8213267846063|
+--------------------+------------